# Imports and Defaults

In [1]:
import json
import os
import zipfile

import bridgestan as bs
from cmdstanpy import CmdStanModel
import numpy as np

In [2]:
seed = 1234
rng = np.random.default_rng(seed)

num_chains = 5
num_samples = 10000

model_name = "stochastic_volatility"
model_path = model_name + ".stan"
data_path = model_name + ".data.json"

# Generate Ref Draws

In [6]:
model = CmdStanModel(stan_file=model_path)
nuts_output = model.sample(
    data=data_path,
    iter_sampling=num_samples,
    chains=num_chains,
    seed=seed,
    thin=10,
    adapt_delta=0.95,
)

ref_draws_np = nuts_output.draws()[:, :, 7:] # [draws, chains, params]
# convert dimension order to [chains, draws, params]
ref_draws_np = np.transpose(ref_draws_np, (1, 0, 2))

23:24:56 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

chain 5 |          | 00:00 Status

23:26:14 - cmdstanpy - INFO - CmdStan done processing.
23:26:14 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: lognormal_lpdf: Scale parameter is inf, but must be positive finite! (in 'stochastic_volatility.stan', line 15, column 4 to column 56)
	Exception: lognormal_lpdf: Scale parameter is inf, but must be positive finite! (in 'stochastic_volatility.stan', line 15, column 4 to column 56)
	Exception: lognormal_lpdf: Scale parameter is -nan, but must be positive finite! (in 'stochastic_volatility.stan', line 15, column 4 to column 56)
Exception: lognormal_lpdf: Scale parameter is inf, but must be positive finite! (in 'stochastic_volatility.stan', line 15, column 4 to column 56)
	Exception: lognormal_lpdf: Scale parameter is inf, but must be positive finite! (in 'stochastic_volatility.stan', line 15, column 4 to column 56)
	Exception: lognormal_lpdf: Scale parameter is inf, but must be positive finite! (in 'stochastic_volatility.stan', line 15, column 4 to column 56

In [7]:
summary = nuts_output.summary()

In [8]:
summary["N_Eff"].describe()

count     504.000000
mean     4841.262579
std       307.642537
min      1492.250000
25%      4725.930000
50%      4873.075000
75%      5014.105000
max      5338.630000
Name: N_Eff, dtype: float64

# Save and Format Reference Draws

In [9]:
# convert from numpy 3d array to json format
num_params = ref_draws_np.shape[-1]
param_names = (
    ["mu", "phi", "sigma"] + [f"tau[{i}]" for i in range(num_params)]
)

ref_draws = []
for chain in ref_draws_np:  # each chain is [n_samples, params_dim]
    chain_dict = {}
    for name, params in zip(param_names, chain.T):
        chain_dict[name] = params.tolist()
    ref_draws.append(chain_dict)

# save to file
with zipfile.ZipFile(f"{model_name}.ref_draws.json.zip", mode="w", compression=zipfile.ZIP_DEFLATED, compresslevel=9) as zip_file: 
    dumped_JSON: str = json.dumps(ref_draws, ensure_ascii=False, indent=4)
    zip_file.writestr(f"{model_name}.ref_draws.json", data=dumped_JSON)